In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools.shell import ShellTool
from dotenv import load_dotenv
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools.shell import ShellTool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


load_dotenv('../../.env')

# Get the prompt to use - you can modify this!
# If you want to see the prompt in full, you can at: https://smith.langchain.com/hub/hwchase17/openai-functions-agent
# prompt = hub.pull("hwchase17/openai-functions-agent")
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You are a helpful GIS agent/consultant.'),
        HumanMessagePromptTemplate.from_template('{input}'),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)
tools = [PythonREPLTool(), ShellTool()]

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [3]:
# path_status = {}
# async for chunk in agent_executor.astream_log(
#     {"input": "what is the weather in sf", "chat_history": []},
#     include_names=["ChatOpenAI"],
# ):
#     for op in chunk.ops:
#         if op["op"] == "add":
#             if op["path"] not in path_status:
#                 path_status[op["path"]] = op["value"]
#             else:
#                 path_status[op["path"]] += op["value"]
#     print(op["path"])
#     print(path_status.get(op["path"]))
#     print("----")

In [15]:
from langchain_core.messages import AIMessageChunk, FunctionMessage, FunctionMessageChunk

tool_calls = {}

# https://github.com/langchain-ai/langchain/discussions/15755#discussioncomment-8071748
# async for chunk in agent_executor.astream_log({"input": "Tell me a story about the user's favorite pet?"}, include_names=['ChatOpenAI']):
async for chunk in agent_executor.astream_log(
    {"input": "List the contents of this folder using both python and bash"},
    include_names=['ChatOpenAI']
):
    for op in chunk.ops:
        print(chunk)
        if op['op'] != 'add':
            continue

        value = op['value']

        if isinstance(value, FunctionMessage):
            print(value.content)

        if not isinstance(value, AIMessageChunk):
            continue

        if 'tool_calls' in value.additional_kwargs:
            tool_call = value.additional_kwargs['tool_calls'][0]
            if tool_call['index'] not in tool_calls:
                tool_calls[tool_call['index']] = {
                    'id': tool_call['id'],
                    'name': tool_call['function']['name'],
                    'arguments': ''
                }
                if len(tool_calls.keys()) > 0:
                    # print('\n')
                    pass
                # print(f'Using tool: {tool_call["function"]["name"]}')
            else: 
                tool_calls[tool_call['index']]['arguments'] += tool_call['function']['arguments']
                # print(tool_call['function']['arguments'], end='')
            continue

        # print(value.content, end='')

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'cee2c45d-41c0-40fb-b1c3-2fc8831cd1d3',
            'logs': {},
            'name': 'AgentExecutor',
            'streamed_output': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI',
  'value': {'end_time': None,
            'final_output': None,
            'id': '2b4e401c-0bce-42db-a8a3-215e3cf6579b',
            'metadata': {},
            'name': 'ChatOpenAI',
            'start_time': '2024-02-07T12:11:44.809+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:3'],
            'type': 'llm'}})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(content='')})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
 

/home/dev/master-thesis/src/langchain/langchain-env/lib/python3.11/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI:2/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content='')})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI:2/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content='')})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': 'Here'},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content='Here')})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': 'Here'},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content='Here')})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' are'},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChu

In [5]:
import json
print(json.dumps(tool_calls, indent=4))

{
    "0": {
        "id": "call_UWA6oHukbuFSxVkL5RkYlvso",
        "name": "Python_REPL",
        "arguments": "{\"__arg1\": \"import os\\nos.listdir()\"}"
    },
    "1": {
        "id": "call_Hex17C7t7uB8l8VkYxEXILfm",
        "name": "terminal",
        "arguments": "{\"commands\": \"ls\"}"
    }
}
